# 🏆 DA-Lab Challenge: The "Extreme Denoising" Solution (Score: 2.096)

# 1. Configuration & Initialization
In this section, we set up the environment, define file paths, and lock the random seed for reproducibility. 

**Key Configurations for Score 2.096:**
* **High-Resolution Text:** Increased `TFIDF_MAX_FEAT` to **10,000** and `NGRAM_RANGE` to **(1, 3)** to capture complex technical jargon and trigrams in the prompts.
* **Latent Dimensions:** Set `SVD_DIM` to **64**, effectively compressing the text into a dense vector space compatible with the provided `embeddinggemma-300m` metric embeddings.
* **Stability:** Used a high estimator count for Random Forest to reduce variance.

In [1]:
# Cell 1: Imports and Configuration
import os, json, random, time, warnings, gc
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls
from lightgbm import LGBMRegressor
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# ---------- CONFIG ----------
KAGGLE_DIR = "/kaggle/input/da5401-2025-data-challenge"
TRAIN_FILE = os.path.join(KAGGLE_DIR, "train_data.json")
TEST_FILE  = os.path.join(KAGGLE_DIR, "test_data.json")
METRIC_NAMES = os.path.join(KAGGLE_DIR, "metric_names.json")
METRIC_EMBS = os.path.join(KAGGLE_DIR, "metric_name_embeddings.npy")
SUBMISSION_FILE = "/kaggle/working/submission.csv"

SEED = 42
N_SPLITS = 5

# --- TWEAK 1: High Resolution Text ---
TFIDF_MAX_FEAT = 10000  # Was 5000. Capture more jargon.
NGRAM_RANGE = (1, 3)    # Was (1, 2). Capture trigrams.

# --- TWEAK 2: Restore Winning SVD ---
SVD_DIM = 64  

# --- TWEAK 3: Keep the stability boost ---
RF_N_ESTIMATORS = 500   # Keep this high for stability.

RIDGE_ALPHA = 1.0
BIAS_ALPHA = 5.0 
NEG_PER_SAMPLE = 1

# LGBM Params
LGBM_PARAMS = {
    "n_estimators": 300,
    "learning_rate": 0.05,
    "num_leaves": 31,
    "random_state": SEED, 
    "n_jobs": -1, "verbosity": -1
}

def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything()
print(f"Config loaded. TFIDF({TFIDF_MAX_FEAT}, {NGRAM_RANGE}) -> SVD({SVD_DIM})")

Config loaded. TFIDF(10000, (1, 3)) -> SVD(64)


# 2. Data Loading & Vectorization
Here we load the JSON datasets and the pre-computed metric embeddings. 

**Preprocessing Strategy:**
1. **Concatenation:** We combine the `prompt`, `system_prompt`, and `response` into a single text block to treat the conversation holistically.
2. **TF-IDF + SVD:** Since we are comparing text fitness against metric definitions, we convert the text into a dense vector space. We use `Sublinear TF` to dampen the effect of very frequent terms.
3. **Dimensionality Reduction:** We apply TruncatedSVD to both the TF-IDF text vectors and the provided metric embeddings to bring them into a shared, comparable 64-dimensional space.

In [2]:
# Cell 2: Load Data and TF-IDF Encoding
print("Loading data...")
with open(TRAIN_FILE, "r", encoding="utf-8") as f: train_raw = json.load(f)
with open(TEST_FILE, "r", encoding="utf-8") as f: test_raw = json.load(f)
with open(METRIC_NAMES, "r", encoding="utf-8") as f: metric_names = json.load(f)
metric_map = {name: i for i, name in enumerate(metric_names)}
metric_embs_raw = np.load(METRIC_EMBS)

def parse_data(raw_data, is_train=True):
    rows = []
    for rec in raw_data:
        rows.append({
            "metric_idx": metric_map.get(rec.get("metric_name"), -1),
            "prompt": (rec.get("prompt","") or ""),
            "system_prompt": (rec.get("system_prompt","") or ""),
            "response": (rec.get("response","") or ""),
            "score": float(rec.get("score", 0.0)) if is_train else 0.0
        })
    return pd.DataFrame(rows)

train_df = parse_data(train_raw, is_train=True)
test_df = parse_data(test_raw, is_train=False)

# Combine text
train_texts = (train_df["prompt"] + " [SEP] " + train_df["system_prompt"] + " [SEP] " + train_df["response"]).tolist()
test_texts = (test_df["prompt"] + " [SEP] " + test_df["system_prompt"] + " [SEP] " + test_df["response"]).tolist()
all_texts = train_texts + test_texts

# --- TWEAK 1: Higher Resolution TF-IDF ---
print(f"Building TF-IDF (max_features={TFIDF_MAX_FEAT}, ngram={NGRAM_RANGE})...")
tfidf = TfidfVectorizer(
    max_features=TFIDF_MAX_FEAT, 
    ngram_range=NGRAM_RANGE,
    sublinear_tf=True,
    min_df=2
)
tfidf.fit(all_texts)
X_tr_tfidf = tfidf.transform(train_texts)
X_te_tfidf = tfidf.transform(test_texts)

# --- TWEAK 2: Restore SVD=64 ---
print(f"Applying TruncatedSVD (n={SVD_DIM})...")
svd_text = TruncatedSVD(n_components=SVD_DIM, random_state=SEED)
svd_text.fit(X_tr_tfidf) 
X_tr_svd = svd_text.transform(X_tr_tfidf)
X_te_svd = svd_text.transform(X_te_tfidf)

print(f"Reducing Metric Embeddings to {SVD_DIM} dims...")
svd_metric = TruncatedSVD(n_components=SVD_DIM, random_state=SEED)
metric_embs_svd = svd_metric.fit_transform(metric_embs_raw)

print("Shapes:", X_tr_svd.shape, metric_embs_svd.shape)

Loading data...
Building TF-IDF (max_features=10000, ngram=(1, 3))...
Applying TruncatedSVD (n=64)...
Reducing Metric Embeddings to 64 dims...
Shapes: (5000, 64) (145, 64)


# 3. Feature Engineering: Metric Learning & Margins
This is the core logic of the solution. [cite_start]Since the goal is to predict "fitness" (similarity), we engineer features based on **geometric distances**.

**Feature Groups:**
* **Prototypes:** We calculate the "centroid" (mean vector) for every metric ID in the training set.
* **Distance Metrics:** We compute Cosine, L1, L2, and Dot Product distances between the text vector and the target metric vector.
* **Margin Features:** Inspired by contrastive learning, we calculate the "Margin." This is the difference between the cosine similarity of the *correct* metric and the *best alternative* metric. A high margin implies the text clearly belongs to the assigned metric.
* **Text Stats:** Ratios of prompt length to response length to capture verbosity patterns.

In [3]:
# Cell 3: Feature Engineering (Prototype Margins)
print("Computing Metric Prototypes from Train Data...")
proto_by_metric = {}
for mid, g in train_df.groupby("metric_idx"):
    idxs = g.index.values
    proto_by_metric[int(mid)] = X_tr_svd[idxs].mean(axis=0)

for i in range(len(metric_names)):
    if i not in proto_by_metric: proto_by_metric[i] = np.zeros(SVD_DIM)

proto_matrix = np.vstack([proto_by_metric[i] for i in range(len(metric_names))])

def pairwise_feats(A, B):
    dot = np.sum(A * B, axis=1)
    an = np.linalg.norm(A, axis=1); bn = np.linalg.norm(B, axis=1)
    cos = dot / (an * bn + 1e-9)
    l1 = np.sum(np.abs(A - B), axis=1)
    l2 = np.sqrt(np.sum((A - B)**2, axis=1))
    prod = A * B; ad = np.abs(A - B)
    return {
        "dot": dot, "cos": cos, "l1": l1, "l2": l2,
        "prod_mean": prod.mean(axis=1), "prod_std": prod.std(axis=1),
        "ad_mean": ad.mean(axis=1), "ad_std": ad.std(axis=1)
    }

def generate_features(df, X_svd):
    row_metrics = np.vstack([metric_embs_svd[i] if i >= 0 else np.zeros(SVD_DIM) for i in df["metric_idx"]])
    row_protos = np.vstack([proto_by_metric.get(int(i), np.zeros(SVD_DIM)) for i in df["metric_idx"]])
    
    feats_metric = pairwise_feats(row_metrics, X_svd)
    feats_proto = pairwise_feats(row_protos, X_svd)
    
    # Margin Calculation
    P_norm = np.linalg.norm(proto_matrix, axis=1) + 1e-9
    T_norm = np.linalg.norm(X_svd, axis=1) + 1e-9
    cos_mat = (X_svd @ proto_matrix.T) / (T_norm[:, None] * P_norm[None, :])
    
    row_idx = np.arange(len(df))
    met_idx = df["metric_idx"].values.astype(int)
    
    self_cos = cos_mat[row_idx, met_idx]
    
    cos_mat_temp = cos_mat.copy()
    cos_mat_temp[row_idx, met_idx] = -1e9
    best_other = cos_mat_temp.max(axis=1)
    margin = self_cos - best_other
    
    out = pd.DataFrame(feats_metric)
    for k, v in feats_proto.items(): out[f"{k}_pt"] = v
    
    out["cos_pt_self"] = self_cos
    out["cos_pt_best_other"] = best_other
    out["cos_pt_margin"] = margin
    
    out["len_prompt"] = df["prompt"].apply(len)
    out["len_response"] = df["response"].apply(len)
    out["ratio_pr"] = (out["len_response"]+1)/(out["len_prompt"]+1)
    
    return out, cos_mat

print("Generating Train Features...")
X_feat_tr, tr_cos_mat = generate_features(train_df, X_tr_svd)
print("Generating Test Features...")
X_feat_te, _ = generate_features(test_df, X_te_svd)

X_feat_tr["score"] = train_df["score"].values
X_feat_tr["metric_idx"] = train_df["metric_idx"].values
X_feat_te["metric_idx"] = test_df["metric_idx"].values

print(f"Feature Matrix: {X_feat_tr.shape}")

Computing Metric Prototypes from Train Data...
Generating Train Features...
Generating Test Features...
Feature Matrix: (5000, 24)


# 4. Negative Sampling (Data Augmentation)
**Challenge:** The competition documentation notes that the training data score distribution is "largely skewed to the high score side". This makes it hard for models to learn what a "bad" response looks like.

**Solution:** We generate synthetic negative samples (`is_synth=True`).
1. We take existing valid rows.
2. We randomly swap the `metric_idx` with a different (incorrect) metric.
3. We assign these mismatched pairs a low score (0, 1, or 2).
4. We re-calculate distance features for these "wrong" pairs.

This teaches the model to penalize semantic mismatches heavily.

In [4]:
# Cell 4: Augmentation
print("Creating Augmented Data...")
aug_rows = []
unique_metrics = train_df["metric_idx"].unique()

for i in tqdm(range(len(train_df))):
    base = X_feat_tr.iloc[i].to_dict()
    base["is_synth"] = False
    aug_rows.append(base)
    
    wrong_idx = int(base["metric_idx"])
    while wrong_idx == base["metric_idx"]:
        wrong_idx = int(random.choice(unique_metrics))
        
    text_vec = X_tr_svd[i].reshape(1, -1)
    wrong_metric_vec = metric_embs_svd[wrong_idx].reshape(1, -1)
    wrong_proto_vec = proto_by_metric[wrong_idx].reshape(1, -1)
    
    pf = pairwise_feats(wrong_metric_vec, text_vec)
    pf_pt = pairwise_feats(wrong_proto_vec, text_vec)
    
    self_cos_wrong = tr_cos_mat[i, wrong_idx]
    cos_row = tr_cos_mat[i].copy()
    cos_row[wrong_idx] = -1e9
    best_other_wrong = cos_row.max()
    margin_wrong = self_cos_wrong - best_other_wrong
    
    neg = base.copy()
    neg["dot"] = pf["dot"][0]; neg["cos"] = pf["cos"][0]; neg["l1"] = pf["l1"][0]; neg["l2"] = pf["l2"][0]
    neg["prod_mean"] = pf["prod_mean"][0]; neg["prod_std"] = pf["prod_std"][0]
    neg["ad_mean"] = pf["ad_mean"][0]; neg["ad_std"] = pf["ad_std"][0]
    
    neg["dot_pt"] = pf_pt["dot"][0]; neg["cos_pt"] = pf_pt["cos"][0]
    neg["l1_pt"] = pf_pt["l1"][0]; neg["l2_pt"] = pf_pt["l2"][0]
    neg["prod_mean_pt"] = pf_pt["prod_mean"][0]; neg["prod_std_pt"] = pf_pt["prod_std"][0]
    neg["ad_mean_pt"] = pf_pt["ad_mean"][0]; neg["ad_std_pt"] = pf_pt["ad_std"][0]
    
    neg["cos_pt_self"] = self_cos_wrong
    neg["cos_pt_best_other"] = best_other_wrong
    neg["cos_pt_margin"] = margin_wrong
    neg["metric_idx"] = wrong_idx
    neg["score"] = float(random.choice([0, 1, 2]))
    neg["is_synth"] = True
    
    aug_rows.append(neg)

aug_df = pd.DataFrame(aug_rows)
print(f"Augmented Data Shape: {aug_df.shape}")

Creating Augmented Data...


  0%|          | 0/5000 [00:00<?, ?it/s]

Augmented Data Shape: (10000, 25)


# 5. Model Training, Stacking & Bias Correction
We use a multi-stage stacking approach to minimize RMSE.

**Stage 1: Augmented Learning**
* We train a **Random Forest** on the dataset *including* the synthetic negative samples. This model specializes in detecting semantic mismatches.

**Stage 2: GroupKFold Stacking**
* We train **Ridge**, **Random Forest**, and **LGBM** on the real data only, using `GroupKFold` to ensure no metric leaks between folds.
* We generate Out-of-Fold (OOF) predictions.

**Stage 3: Ensemble & Bias Correction**
* **NNLS (Non-Negative Least Squares):** We learn the optimal linear combination of the base models.
* **Bayesian Bias Correction:** We calculate the mean error per metric group and apply a smoothed offset. This corrects systematic over/under-prediction for specific metric types.

In [5]:
# Cell 5: Model Training & Stacking
feat_cols = [c for c in aug_df.columns if c not in ["score", "metric_idx", "is_synth"]]
X = aug_df[feat_cols].fillna(0).values
y = aug_df["score"].values

print(f"Training Augmented Random Forest (n={RF_N_ESTIMATORS})...")
rf_aug = RandomForestRegressor(n_estimators=RF_N_ESTIMATORS, n_jobs=-1, random_state=SEED)
rf_aug.fit(X, y)

print("Training Base Models & Generating OOFs...")
X_real = X_feat_tr[feat_cols].fillna(0).values
y_real = X_feat_tr["score"].values
groups_real = X_feat_tr["metric_idx"].values

gkf = GroupKFold(n_splits=N_SPLITS)
oof_ridge = np.zeros(len(y_real))
oof_rf = np.zeros(len(y_real))
oof_lgbm = np.zeros(len(y_real))
aug_preds_oof = rf_aug.predict(X_real)

for fold, (tr_idx, val_idx) in enumerate(gkf.split(X_real, y_real, groups_real)):
    X_tr, y_tr = X_real[tr_idx], y_real[tr_idx]
    X_val = X_real[val_idx]
    
    ridge = Ridge(alpha=RIDGE_ALPHA, random_state=SEED)
    ridge.fit(X_tr, y_tr)
    oof_ridge[val_idx] = ridge.predict(X_val)
    
    rf = RandomForestRegressor(n_estimators=RF_N_ESTIMATORS, n_jobs=-1, random_state=SEED)
    rf.fit(X_tr, y_tr)
    oof_rf[val_idx] = rf.predict(X_val)

    lgbm = LGBMRegressor(**LGBM_PARAMS)
    lgbm.fit(X_tr, y_tr)
    oof_lgbm[val_idx] = lgbm.predict(X_val)

print(f"Ridge OOF: {mean_squared_error(y_real, oof_ridge, squared=False):.4f}")
print(f"RF OOF:    {mean_squared_error(y_real, oof_rf, squared=False):.4f}")
print(f"LGBM OOF:  {mean_squared_error(y_real, oof_lgbm, squared=False):.4f}")

print("Stacking...")
X_stack = np.vstack([oof_ridge, oof_rf, oof_lgbm, aug_preds_oof]).T
coef, _ = nnls(X_stack, y_real)
print(f"NNLS Coeffs [Ridge, RF, LGBM, AugRF]: {coef}")
meta_oof = X_stack.dot(coef)

print(f"Applying Bias Correction (Alpha={BIAS_ALPHA})...")
temp_df = pd.DataFrame({"metric_idx": groups_real, "score": y_real, "pred": meta_oof})
bias_stats = temp_df.groupby("metric_idx").agg(
    n=("score", "size"), mean_true=("score", "mean"), mean_pred=("pred", "mean")
).reset_index()
bias_stats["bias"] = (bias_stats["n"] * (bias_stats["mean_true"] - bias_stats["mean_pred"])) / (bias_stats["n"] + BIAS_ALPHA)
bias_map = bias_stats.set_index("metric_idx")["bias"].to_dict()

meta_oof_corr = meta_oof + temp_df["metric_idx"].map(bias_map).fillna(0.0)
meta_oof_corr = np.clip(meta_oof_corr, 0.0, 10.0)

print(f">>> Final OOF RMSE: {mean_squared_error(y_real, meta_oof_corr, squared=False):.4f} <<<")

Training Augmented Random Forest (n=500)...
Training Base Models & Generating OOFs...
Ridge OOF: 0.9420
RF OOF:    0.9730
LGBM OOF:  0.9928
Stacking...
NNLS Coeffs [Ridge, RF, LGBM, AugRF]: [0.16440274 0.03906174 0.         0.82881879]
Applying Bias Correction (Alpha=5.0)...
>>> Final OOF RMSE: 0.5986 <<<


# 6. Final Inference & Submission
In this final step, we retrain our base models on the **full** training dataset to maximize information gain.

**Steps:**
1. Retrain Ridge, RF, and LGBM on all real data.
2. Apply the stacked weights (NNLS coefficients) derived from the OOF validation.
3. Apply the Bias Correction map.
4. **Clip Results:** Ensure all predictions fall within the valid `[0, 10]` range.

In [6]:
# Cell 6: Test Submission
print("Generating Test Predictions...")
X_test_real = X_feat_te[feat_cols].fillna(0).values

print("Retraining base models on full data...")
ridge_full = Ridge(alpha=RIDGE_ALPHA, random_state=SEED)
ridge_full.fit(X_real, y_real)
test_ridge = ridge_full.predict(X_test_real)

rf_full = RandomForestRegressor(n_estimators=RF_N_ESTIMATORS, n_jobs=-1, random_state=SEED)
rf_full.fit(X_real, y_real)
test_rf = rf_full.predict(X_test_real)

lgbm_full = LGBMRegressor(**LGBM_PARAMS)
lgbm_full.fit(X_real, y_real)
test_lgbm = lgbm_full.predict(X_test_real)

test_aug = rf_aug.predict(X_test_real)

# Stack
X_test_stack = np.vstack([test_ridge, test_rf, test_lgbm, test_aug]).T
test_meta = X_test_stack.dot(coef)

# Bias Correct
test_biases = X_feat_te["metric_idx"].map(bias_map).fillna(0.0).values
test_final = np.clip(test_meta + test_biases, 0.0, 10.0)

# Save
submission = pd.DataFrame({
    "ID": test_df.index + 1 if "ID" not in test_raw[0] else [r.get("ID") for r in test_raw],
    "score": test_final
})

try:
    sample = pd.read_csv(os.path.join(KAGGLE_DIR, "sample_submission.csv"))
    if "ID" in sample.columns:
        submission["ID"] = sample["ID"]
except: pass

submission.to_csv(SUBMISSION_FILE, index=False)
print(f"Saved to {SUBMISSION_FILE}")
print(submission.head())

Generating Test Predictions...
Retraining base models on full data...
Saved to /kaggle/working/submission.csv
   ID     score
0   1  8.393370
1   2  9.443199
2   3  9.042131
3   4  9.335973
4   5  2.579259
